In [6]:
#! pip install -U climetlab --quiet
#! pip install -U climetlab_s2s_ai_challenge --quiet

In [2]:
import climetlab as cml 

/home/mafp/.conda/envs/dev8/lib/python3.8/site-packages/ecmwflibs.libs/libMagPlus-887d8f5c.so


In [3]:
import climetlab_s2s_ai_challenge
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-challenge plugin version : {climetlab_s2s_ai_challenge.__version__}')

Climetlab version : 0.5.6
Climetlab-s2s-ai-challenge plugin version : 0.4.16


# Observations data for training

In [4]:
cmlds = cml.load_dataset("s2s-ai-challenge-observations",
                         date=20200102,
                         parameter='tp'
                       )

By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data.  This dataset has been dowloaded from IRIDL.


This climetlab dataset can be used as a xarray.Dataset or as a pandas.DataFrame :

In [5]:
cmlds.to_xarray()

<xarray.Dataset>
Dimensions:        (forecast_time: 1, latitude: 19, lead_time: 3, longitude: 36, realization: 3)
Coordinates:
  * realization    (realization) int64 0 1 2
  * forecast_time  (forecast_time) datetime64[ns] 2020-01-02
  * lead_time      (lead_time) timedelta64[ns] 1 days 2 days 3 days
  * latitude       (latitude) float64 90.0 80.0 70.0 60.0 ... -70.0 -80.0 -90.0
  * longitude      (longitude) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0
    valid_time     (forecast_time, lead_time) datetime64[ns] ...
Data variables:
    tp             (realization, forecast_time, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-04-29T16:26 GRIB to CDM+CF via cfgrib-0.9.9...